In [38]:
import ee
import geemap
import pandas as pd

In [39]:
Map = geemap.Map()

In [40]:
def registros(cod, m2, ha):
    diccionario = {}
    diccionario["COMUNA"] = cod
    diccionario[year + str("_m2")] = m2
    diccionario[year + str("_ha")] = ha

    return diccionario

In [41]:
df = pd.read_excel("SUBC.xlsx")
comunas = df["COD_SUBC"].unique().tolist()

In [42]:
startDate = ['2001-01-01', '2002-01-01', '2003-01-01', '2004-01-01', '2005-01-01', '2006-01-01', '2007-01-01', '2008-01-01', '2009-01-01', '2010-01-01', '2011-01-01', '2012-01-01', '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01']
endDate = ['2002-01-01', '2003-01-01', '2004-01-01', '2005-01-01', '2006-01-01', '2007-01-01', '2008-01-01', '2009-01-01', '2010-01-01', '2011-01-01', '2012-01-01', '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01']

In [26]:
for j in zip(startDate,endDate):
    
    Fecha_inicial = str(j[0])
    Fecha_final = str(j[1])
    
    year = Fecha_inicial[:4]
    
    # year_text_m2 = year
    # year_text_ha = year
    
    print(year)
    puntos = []

    for i in comunas:
        studyArea = ee.FeatureCollection("users/testHector/Subsubcuencas_v2")
        studyArea = studyArea.filterMetadata("COD_SUBC","equals", str(i).zfill(4))

        dataset = ee.ImageCollection('MODIS/006/MCD64A1').filter(ee.Filter.date(Fecha_inicial, Fecha_final))
        image = dataset.sum()


        mask = image.select(0).mask().rename('mask')
        area = ee.Image.pixelArea().multiply(mask).rename('area')


        #sumDictionarypolygon1 = mask.addBands(area).reduceRegion({
        sumDictionarypolygon1 = area.reduceRegion(
          reducer = ee.Reducer.sum(),
          geometry = studyArea.geometry(),
          scale = 500,
          maxPixels = 1e9
        )

        # print('sum for pol 1 mask, scale=500', sumDictionarypolygon1.values(obj))
        area_m2 = sumDictionarypolygon1.getInfo()
        
        aQuemada_m2 = float(area_m2['area'])
        aQuemada_ha = float(area_m2['area']) / 10000

        # print(aQuemada_m2)
        # print(aQuemada_ha)

        diccionario = registros(i, aQuemada_m2, aQuemada_ha)
        puntos.append(diccionario.copy())
        
    data = pd.DataFrame(puntos)
    data.to_excel("subc_ha_quemada_anual/" + str(year) + ".xlsx", index=False)

2011
2012
2013
2014
2015
